In [ ]:
'''
    need to open main list
    
    run the titles through something to see if it has names in the title
    
    (see if the names are political figures?)
    
    output articles with names in title to NameList
    
    run titles (possibly whole article) through NLP to see what the main point of the article is
    
    put articles into individual files whose name is the main point
'''

In [49]:
import os
import pickle
import nltk
from nltk.tokenize import PunktSentenceTokenizer
from nltk.corpus import state_union

def findName(line):
    
    train_text = state_union.raw("2005-GWBush.txt")
    
    custom_sent_tokenizer = PunktSentenceTokenizer(train_text)
    
    #text = inputFile.read()
    
    # tokenizing the input file (the main file)
    tokenized = custom_sent_tokenizer.tokenize(line)[0]
    #print(tokenized)
    
    # finding the names
    try:
        words = nltk.word_tokenize(tokenized)
        tagged = nltk.pos_tag(words)
        namedEnt = nltk.ne_chunk(tagged, binary=True)
        #print(namedEnt)
        #print("Broken down: ")
        # determining if there's a name in the title
        # that's such a broad ass descpritor (NNP)
        for ent in namedEnt:
            for entType in ent:
                if entType[1] == "NNP":
                    if entType[0] != "CNN":
                        print("Found:")
                        print(line)
                        print(entType[0])
                        print("\n\n")
                        return True
                    else:
                        return False
    except:
        print("Error with tokenizing")
        return False
        
    return False


# add the title and url to the main file
def addOutputFile(titleLine, urlLine):
    outputFile = open("NEList.txt", "a")
    outputFile.seek(0, os.SEEK_END)
    
    #print(titleLine)
    #print(urlLine)
    
    toWrite = [titleLine, urlLine]
    
    outputFile.writelines(toWrite)
    #outputFile.write(urlLine)
    outputFile.close()

    
    

    
    
    
    
print("Starting...")
    
inputFile = open("MainList.txt", "r")

inputFile.seek(0, os.SEEK_SET)

line = inputFile.readline()

print("Finding name...")
while line:

    #print(line)

    if findName(line):
        # if a name is found in the title, write it out to the new file
        titleLine = line
        urlLine = inputFile.readline()
        addOutputFile(titleLine, urlLine)
    else:
        # otherwise, go the next title
        line = inputFile.readline()
    
    line = inputFile.readline()

inputFile.close()

print("Done!")

Starting...
Finding name...
Found:
Iraq Peshmerga fighters arrive in embattled Kobani

Iraq



Found:
100 feared trapped in India landslide debris

India



Found:
Carter criticizes Obama on ISIS: 'We waited too long'

Carter



Found:
"Hong Kong's youth arm with cell phones, umbrellas"

Hong



Found:
'Big Bang Theory' star Kaley Cuoco weds

Bang



Found:
Husband of jailed Sudanese Christian fears for family's lives

Husband



Found:
"Oscar Pistorius loved Reeva Steenkamp, agent says"

Reeva



Found:
Kristin Chenoweth to play Maleficent

Kristin



Found:
Amanda Kloots to join 'The Talk' as co-host alongside Elaine Welteroth - CNN

Amanda



Found:
The TSA race to the gate | CNN Travel

TSA



Found:
Robert Joe of 'I Wouldn't Go in There': Asia's scariest places | CNN Travel

Robert



Found:
"In Albuquerque, protesters clash with police"

Albuquerque



Found:
Harvard approves student group devoted to kinky sex on campus

Harvard



Found:
"Omarosa Manigault Newman, ex-'Apprentice